# __Regularizacion en Modelos Lineales__ 

La mejor manera de reducir el sobreajuste en un modelo lineal es con la regularizacion. Este proceso consiste en restringir los pesos del modelo. Existen tres formas de restringir los pesos como veremos a continuacion:

## Regresion Ridge

Es una version regularizada de la regresion lineal y consiste en agregar el termino $\alpha \sum_{i}^{n} \theta_{i}^2$ a la funcion de costo. Esto obliga al algoritmo de aprendizaje a no solo ajustar los datos, sino tambien a mantener los pesos del modelo los mas pequeno posible.

- Este termino solo debe agregarse a la funcion de costo durante el entrenamiento. Una vez entrenado, se evalua el rendimiento del modelo utilizando una medida no regularizada.

- Es comun que la funcion de costo usada durante el entrenamiento sea diferente de la medida de rendimiento del modelo.

- La funcion de costo utilizada en el entrenamiento debe tener derivadas amigables para la optimizacion mientras que la medida de rendimiento debe estar lo mas cerca posible del objetivo final.

- El parametro $\alpha$ controla cuanto desea regularizar el modelo. Si $\alpha=0$, la regresion Ridge se vuelve una regresion lineal. Si $\alpha$ es muy grande, todos los pesos terminan muy cerca de cero y el resultado es una linea plana que pasa por la media de los datos.

- Es importante escalar los datos con _StandarScaler_ antes de alicar la regularizacion ya que es sensible a la escala de las caracteristicas de entrada.